In [ ]:
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from torchvision import transforms

# TODO #1: Define a transform to pre-process the testing images.
# defaulty provided transformation I have used in training
transform_test = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

#-#-#-# Do NOT modify the code below this line. #-#-#-#

# Create the data loader.
data_loader = get_loader(transform=transform_test,    
                         mode='test')

In [ ]:
# Watch for any changes in model.py, and re-load it automatically.
% load_ext autoreload
% autoreload 2

import os
import torch
from model import EncoderCNN, DecoderRNN

# TODO #2: Specify the saved models to load.
encoder_file = 'encoder-3.pkl'
decoder_file = 'decoder-3.pkl'

# TODO #3: Select appropriate values for the Python variables below.
embed_size = 128
hidden_size = 512

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)
print('vocab_size:',vocab_size)

# Initialize the encoder and decoder, and set each to inference mode.
encoder = EncoderCNN(embed_size)
encoder.eval()
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
decoder.eval()

# Load the trained weights.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))

# Move models to GPU if CUDA is available.
encoder.to(device)
decoder.to(device)

In [ ]:
# TODO #4: Complete the function.

###
### my implementation of clean_sentence method, which I have asked assistance for :
### https://knowledge.udacity.com/questions/317345

# def clean_sentence(output):
#     sentence_list = []
#     # getting idx2word and removing <start>, <end> and <unk> so it won't be a part of sentences
#     dictionary = list(data_loader.dataset.vocab.idx2word.items())[3:-1]
#     dictionary = dict(dictionary)
#     #print(dictionary)
#     print('output:',output)
    
#     for word_index in output:
#         #print(word_index)
#         for word_index_dictionary in dictionary:
#             if word_index == word_index_dictionary:
#                 #print(dictionary[word_index_dictionary])
#                 sentence_list.append(dictionary[word_index_dictionary])
    
#     sentence = ' '.join(sentence_list)
    
#     return sentence



###
### pseudo code that I was pointed at by mentor :

#def clean_sentence(output):
    #get the end token
    #get the start token
    #iterate for each token in the output:
         #Check if the token is a start_token:
              #If it is continue iterating
         #Check if the token is end_token:
              #if it is break
         #keep accumulating all the words in a variable and a space to them to form a sentence
    #Return the accumulator variable

    
def clean_sentence(output):
    sentence_list = []
    end_token = data_loader.dataset.vocab.word2idx['<end>']
    start_token = data_loader.dataset.vocab.word2idx['<start>']
    
    #print('start_token:',start_token)
    #print('end_token:',end_token)
    
    for specific_token in output:
        #print('specific_token:',specific_token)
        if specific_token != end_token:
            sentence_list.append(data_loader.dataset.vocab.idx2word[specific_token])
        if specific_token == end_token:
            break
            
    
    sentence_list = sentence_list[1:]
    
    sentence = ' '.join(sentence_list)
    return sentence
        
